<a href="https://colab.research.google.com/github/nastuaah/FaceRecognition/blob/main/OpenCV%2BDlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
import cv2
import dlib

# Монтируем Google Drive
drive.mount('/content/drive')

# Загружаем предобученную модель для обнаружения ключевых точек лица из dlib
predictor_path = "/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Счетчик для подсчета количества поджатых губ
lip_press_count = 0

# Путь к директории с датасетом на Google Drive
dataset_path = "/content/drive/MyDrive/lips_dataset"

# Обработка изображений в датасете
for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    img = cv2.imread(img_path)

    # Конвертируем изображение в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Обнаруживаем лица на изображении
    faces = detector(gray)

    for face in faces:
        # Обнаруживаем ключевые точки лица
        landmarks = predictor(gray, face)

        # Извлекаем координаты точек в области рта
        mouth_points = []
        for n in range(48, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            mouth_points.append((x, y))

        # Вычисляем расстояние между верхней и нижней губой
        lip_distance = mouth_points[15][1] - mouth_points[16][1]

        # Если расстояние меньше порогового значения, считаем, что губы поджаты
        if lip_distance < 1:
            lip_press_count += 1

# Выводим количество раз, когда губы были поджаты
print(f"Total lip presses: {lip_press_count}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total lip presses: 28


In [3]:
import os
from google.colab import drive
import cv2
import numpy as np

# Монтируем Google Drive
drive.mount('/content/drive')

# Переменные для подсчета количества взглядов в каждую сторону
forward_look_count = 0
left_look_count = 0
right_look_count = 0

# Функция для определения направления взгляда
def determine_gaze_direction(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    left_eye_centers = []
    right_eye_centers = []

    for (ex, ey, ew, eh) in eyes:
        eye_center_x = ex + ew // 2
        eye_center_y = ey + eh // 2

        if eye_center_x < frame.shape[1] // 2:
            left_eye_centers.append((eye_center_x, eye_center_y))
        elif eye_center_x > frame.shape[1] // 2:
            right_eye_centers.append((eye_center_x, eye_center_y))

    if len(left_eye_centers) > len(right_eye_centers):
        return 'LEFT'
    elif len(right_eye_centers) > len(left_eye_centers):
        return 'RIGHT'
    else:
        return 'FORWARD'

# Загрузка каскада Хаара для глаз
eye_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/haarcascade_eye.xml')

# Обработка датасета
dataset_path = '/content/drive/MyDrive/eye_dataset1'

for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    img = cv2.imread(img_path)
    gaze_direction = determine_gaze_direction(img)

    if gaze_direction == 'FORWARD':
        forward_look_count += 1
    elif gaze_direction == 'LEFT':
        left_look_count += 1
    elif gaze_direction == 'RIGHT':
        right_look_count += 1
    else:
      "NO EYES"

# Вывод результатов
print(f'Number of forward looks: {forward_look_count}')
print(f'Number of left looks: {left_look_count}')
print(f'Number of right looks: {right_look_count}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of forward looks: 9
Number of left looks: 14
Number of right looks: 8


In [1]:
import os
from google.colab import drive
import cv2
import dlib

# Монтируем Google Drive
drive.mount('/content/drive')

# Загружаем предобученную модель для обнаружения ключевых точек лица из dlib
predictor_path = "/content/drive/MyDrive/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Переменные для подсчета количества наклонов головы
head_left_count = 0
head_right_count = 0
head_straight_count = 0

# Путь к директории с датасетом на Google Drive
dataset_path = "/content/drive/MyDrive/head_dataset"

# Обработка изображений в датасете
for filename in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, filename)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Обнаруживаем лица на изображении
    faces = detector(gray)

    for face in faces:
        # Обнаруживаем ключевые точки лица
        landmarks = predictor(gray, face)

        # Извлекаем координаты левого и правого глаза
        left_eye = landmarks.parts()[36:42]
        right_eye = landmarks.parts()[42:48]

        # Вычисляем угол наклона головы
        left_eye_center = (sum([p.x for p in left_eye]) / len(left_eye),
                           sum([p.y for p in left_eye]) / len(left_eye))
        right_eye_center = (sum([p.x for p in right_eye]) / len(right_eye),
                            sum([p.y for p in right_eye]) / len(right_eye))

        dx = right_eye_center[0] - left_eye_center[0]
        dy = right_eye_center[1] - left_eye_center[1]
        head_pose_angle = cv2.fastAtan2(dy, dx)

        # Определяем направление наклона головы
        if head_pose_angle > 15:
            head_left_count += 1
        elif head_pose_angle < -15:
            head_right_count += 1
        else:
            head_straight_count += 1

# Вывод результатов
print(f'Number of times head tilted left: {head_left_count}')
print(f'Number of times head tilted right: {head_right_count}')
print(f'Number of times head straight: {head_straight_count}')



Mounted at /content/drive
Number of times head tilted left: 12
Number of times head tilted right: 15
Number of times head straight: 10
